In [2]:
!pip install -qU openai-agents "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [166]:
# Just for Colab to run async methods
import nest_asyncio
nest_asyncio.apply()

In [167]:
from google.colab import userdata
import os

API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GEMINI_API_KEY"] = API_KEY

In [168]:
from agents import Agent, Runner, set_tracing_disabled, GuardrailFunctionOutput, input_guardrail, output_guardrail, RunContextWrapper, TResponseInputItem, InputGuardrailTripwireTriggered
from agents.extensions.models.litellm_model import LitellmModel
from pydantic import BaseModel
import asyncio

In [169]:
class MathHomeWorkOutput(BaseModel):
    is_not_math_related_task: bool
    rasoning: str

In [170]:
set_tracing_disabled(True)

In [171]:
guardrail_agent = Agent(
    name= "Guardrail Check",
    instructions= "Check if input not Math related task",
    model=LitellmModel(model="gemini/gemini-1.5-flash"),
    output_type=MathHomeWorkOutput
)

In [172]:
@input_guardrail
async def math_guardrail( ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]) -> GuardrailFunctionOutput:

  result = await Runner.run(guardrail_agent, input, context=ctx.context)
  print(result.final_output.model_dump())

  return GuardrailFunctionOutput(
      output_info=result.final_output,
      tripwire_triggered= result.final_output.is_not_math_related_task,
  )


In [173]:
assistant_agent = Agent(
      name="Assistant",
      instructions="You are a helpful asistant agent of solving Math related.",
      model=LitellmModel(model="gemini/gemini-2.0-flash"), # Good model use for better result,
      input_guardrails=[math_guardrail]
  )

In [174]:
try:
  main_agent_result = Runner.run_sync(assistant_agent, "Hello")
  print(main_agent_result.final_output)
except InputGuardrailTripwireTriggered as e:
  print("Please ask only math related question.")

{'is_not_math_related_task': True, 'rasoning': "The input is a greeting and doesn't contain any mathematical expressions or questions."}
Please ask only math related question.
